In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# IBM Data Science Specialization Capstone\n",
    "This notebook is part of the IBM datascience capstone projects.\n",
    "\n",
    "## Segmenting and Clustering Neighborhoods in Toronto - Part 2"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "In Part 1 of this project, we collected the Toronto neighborhoods data from a web page, converted it to a pandas dataframe, and cleaned and pre-processed it.\n",
    "\n",
    "In this notebook, we will get the latitude and logitude of each neighborhood and add it to the dataframe so that we can utilize the Foursquare location data."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Install required packages"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Requirement already satisfied: pandas in /usr/local/lib/python3.7/site-packages (0.24.2)\n",
      "Requirement already satisfied: numpy>=1.12.0 in /usr/local/lib/python3.7/site-packages (from pandas) (1.16.4)\n",
      "Requirement already satisfied: pytz>=2011k in /usr/local/lib/python3.7/site-packages (from pandas) (2019.1)\n",
      "Requirement already satisfied: python-dateutil>=2.5.0 in /usr/local/lib/python3.7/site-packages (from pandas) (2.8.0)\n",
      "Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.7/site-packages (from python-dateutil>=2.5.0->pandas) (1.12.0)\n",
      "Requirement already satisfied: beautifulsoup4 in /usr/local/lib/python3.7/site-packages (4.7.1)\n",
      "Requirement already satisfied: soupsieve>=1.2 in /usr/local/lib/python3.7/site-packages (from beautifulsoup4) (1.9.1)\n",
      "Requirement already satisfied: requests in /usr/local/lib/python3.7/site-packages (2.22.0)\n",
      "Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/local/lib/python3.7/site-packages (from requests) (3.0.4)\n",
      "Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/site-packages (from requests) (1.25.3)\n",
      "Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/site-packages (from requests) (2019.3.9)\n",
      "Requirement already satisfied: idna<2.9,>=2.5 in /usr/local/lib/python3.7/site-packages (from requests) (2.8)\n"
     ]
    }
   ],
   "source": [
    "import sys\n",
    "!{sys.executable} -m pip install pandas\n",
    "!{sys.executable} -m pip install beautifulsoup4\n",
    "!{sys.executable} -m pip install requests"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Import dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "from bs4 import BeautifulSoup\n",
    "import requests\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 1 Revisited - Collect neighborhood data for Toronto and convert it into a pandas dataframe"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The data will be collected from Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M\n",
    "\n",
    "This page contains a list of postal codes in Canada where the first letter is M."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the HTML text from the wiki page\n",
    "wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'\n",
    "html_text = requests.get(wiki_url).text\n",
    "\n",
    "# Extract the table data using BeautifulSoup\n",
    "soup = BeautifulSoup(html_text)\n",
    "table = soup.find('table', attrs={'class':'wikitable sortable'})\n",
    "trs = table.find_all('tr')\n",
    "\n",
    "# Extract the text from all the table cells and add all rows\n",
    "# to a list of rows.\n",
    "rows = list()\n",
    "for tr in trs:\n",
    "    td = tr.find_all('td')\n",
    "    row = [ele.text.strip() for ele in td]\n",
    "    if row:\n",
    "        # Ignore empty rows with no 'td',\n",
    "        # applicable for the column headers row.\n",
    "        rows.append(row)\n",
    "        \n",
    "# Convert the data to a pandas dataframe with \n",
    "# columns 'PostalCode', 'Borough', and 'Neighborhood'.\n",
    "df = pd.DataFrame(rows,\n",
    "                  columns=['PostalCode', 'Borough', 'Neighborhood'])\n",
    "\n",
    "# Remove rows with column 'Not assigned' for column 'Borough'\n",
    "df = df[df.Borough != 'Not assigned']\n",
    "df.reset_index(inplace=True, drop=True)\n",
    "\n",
    "# If a cell has a borough but a Not assigned neighborhood, \n",
    "# then the neighborhood will be the same as the borough.\n",
    "# So, replace 'Neighborhood' columns with value as\n",
    "# 'Not assigned' with the value of its 'Borough'\n",
    "df['Neighborhood'] = df.apply(\n",
    "    lambda row: \n",
    "    row['Borough'] if row['Neighborhood'] == 'Not assigned' \n",
    "    else row['Neighborhood'],\n",
    "    axis=1)\n",
    "\n",
    "# More than one neighborhood can exist in one postal code area.\n",
    "# Combine rows with the same postal code into a single row.\n",
    "# For example, in the table on the Wikipedia page, M5A is \n",
    "# listed twice and has two neighborhoods: Harbourfront and\n",
    "# Regent Park. These two rows will be combined into one row\n",
    "# with the neighborhoods separated with a comma.\n",
    "df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\\\n",
    "    apply(', '.join).to_frame()\n",
    "df.reset_index(inplace=True)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>PostalCode</th>\n",
       "      <th>Borough</th>\n",
       "      <th>Neighborhood</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>M1B</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Rouge, Malvern</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>M1C</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Highland Creek, Rouge Hill, Port Union</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>M1E</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Guildwood, Morningside, West Hill</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>M1G</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Woburn</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>M1H</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cedarbrae</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>M1J</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Scarborough Village</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>M1K</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>East Birchmount Park, Ionview, Kennedy Park</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>M1L</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Clairlea, Golden Mile, Oakridge</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>M1M</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cliffcrest, Cliffside, Scarborough Village West</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>M1N</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Birch Cliff, Cliffside West</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  PostalCode      Borough                                     Neighborhood\n",
       "0        M1B  Scarborough                                   Rouge, Malvern\n",
       "1        M1C  Scarborough           Highland Creek, Rouge Hill, Port Union\n",
       "2        M1E  Scarborough                Guildwood, Morningside, West Hill\n",
       "3        M1G  Scarborough                                           Woburn\n",
       "4        M1H  Scarborough                                        Cedarbrae\n",
       "5        M1J  Scarborough                              Scarborough Village\n",
       "6        M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park\n",
       "7        M1L  Scarborough                  Clairlea, Golden Mile, Oakridge\n",
       "8        M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West\n",
       "9        M1N  Scarborough                      Birch Cliff, Cliffside West"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df.head(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(103, 3)"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df.shape"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 2 - Add latitude and longitude data to the dataframe"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Get latitude and logitude data"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We will be using the file https://cocl.us/Geospatial_data to get the data.\n",
    "\n",
    "Read the CSV data to a pandas dataframe."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now let us explore the data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Geospatial dataframe's shape:  (103, 3)\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Postal Code</th>\n",
       "      <th>Latitude</th>\n",
       "      <th>Longitude</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>M1B</td>\n",
       "      <td>43.806686</td>\n",
       "      <td>-79.194353</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>M1C</td>\n",
       "      <td>43.784535</td>\n",
       "      <td>-79.160497</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>M1E</td>\n",
       "      <td>43.763573</td>\n",
       "      <td>-79.188711</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>M1G</td>\n",
       "      <td>43.770992</td>\n",
       "      <td>-79.216917</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>M1H</td>\n",
       "      <td>43.773136</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  Postal Code   Latitude  Longitude\n",
       "0         M1B  43.806686 -79.194353\n",
       "1         M1C  43.784535 -79.160497\n",
       "2         M1E  43.763573 -79.188711\n",
       "3         M1G  43.770992 -79.216917\n",
       "4         M1H  43.773136 -79.239476"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print(\"Geospatial dataframe's shape: \", geospatial_data.shape)\n",
    "geospatial_data.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The size (103 rows) matches that of our original dataframe df.\n",
    "\n",
    "### Combine the two dataframes\n",
    "\n",
    "Now let us combine the two dataframes such that the latitude and logitude values from 'geospatial_data' are assigned to their corresponding rows in our original dataframe, 'df'."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>Borough</th>\n",
       "      <th>Neighborhood</th>\n",
       "      <th>Latitude</th>\n",
       "      <th>Longitude</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>M1B</th>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Rouge, Malvern</td>\n",
       "      <td>43.806686</td>\n",
       "      <td>-79.194353</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>M1C</th>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Highland Creek, Rouge Hill, Port Union</td>\n",
       "      <td>43.784535</td>\n",
       "      <td>-79.160497</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>M1E</th>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Guildwood, Morningside, West Hill</td>\n",
       "      <td>43.763573</td>\n",
       "      <td>-79.188711</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>M1G</th>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Woburn</td>\n",
       "      <td>43.770992</td>\n",
       "      <td>-79.216917</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>M1H</th>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cedarbrae</td>\n",
       "      <td>43.773136</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "         Borough                            Neighborhood   Latitude  Longitude\n",
       "M1B  Scarborough                          Rouge, Malvern  43.806686 -79.194353\n",
       "M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535 -79.160497\n",
       "M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573 -79.188711\n",
       "M1G  Scarborough                                  Woburn  43.770992 -79.216917\n",
       "M1H  Scarborough                               Cedarbrae  43.773136 -79.239476"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Concatenate the two dataframes using column \"PostalCode\" from 'df' and\n",
    "# \"Postal Code\" from 'geospatial_data'.\n",
    "df = pd.concat(\n",
    "    [df.set_index('PostalCode'), geospatial_data.set_index('Postal Code')],\n",
    "    axis=1, join='inner')\n",
    "df.head()\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Postal code has been set as the index, change it to a non-index column and reset index."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>index</th>\n",
       "      <th>Borough</th>\n",
       "      <th>Neighborhood</th>\n",
       "      <th>Latitude</th>\n",
       "      <th>Longitude</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>M1B</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Rouge, Malvern</td>\n",
       "      <td>43.806686</td>\n",
       "      <td>-79.194353</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>M1C</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Highland Creek, Rouge Hill, Port Union</td>\n",
       "      <td>43.784535</td>\n",
       "      <td>-79.160497</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>M1E</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Guildwood, Morningside, West Hill</td>\n",
       "      <td>43.763573</td>\n",
       "      <td>-79.188711</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>M1G</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Woburn</td>\n",
       "      <td>43.770992</td>\n",
       "      <td>-79.216917</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>M1H</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cedarbrae</td>\n",
       "      <td>43.773136</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  index      Borough                            Neighborhood   Latitude  \\\n",
       "0   M1B  Scarborough                          Rouge, Malvern  43.806686   \n",
       "1   M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   \n",
       "2   M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   \n",
       "3   M1G  Scarborough                                  Woburn  43.770992   \n",
       "4   M1H  Scarborough                               Cedarbrae  43.773136   \n",
       "\n",
       "   Longitude  \n",
       "0 -79.194353  \n",
       "1 -79.160497  \n",
       "2 -79.188711  \n",
       "3 -79.216917  \n",
       "4 -79.239476  "
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df.reset_index(inplace=True)\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Rename the 'index' column to 'PostalCode'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>PostalCode</th>\n",
       "      <th>Borough</th>\n",
       "      <th>Neighborhood</th>\n",
       "      <th>Latitude</th>\n",
       "      <th>Longitude</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>M1B</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Rouge, Malvern</td>\n",
       "      <td>43.806686</td>\n",
       "      <td>-79.194353</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>M1C</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Highland Creek, Rouge Hill, Port Union</td>\n",
       "      <td>43.784535</td>\n",
       "      <td>-79.160497</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>M1E</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Guildwood, Morningside, West Hill</td>\n",
       "      <td>43.763573</td>\n",
       "      <td>-79.188711</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>M1G</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Woburn</td>\n",
       "      <td>43.770992</td>\n",
       "      <td>-79.216917</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>M1H</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cedarbrae</td>\n",
       "      <td>43.773136</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>M1J</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Scarborough Village</td>\n",
       "      <td>43.744734</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>M1K</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>East Birchmount Park, Ionview, Kennedy Park</td>\n",
       "      <td>43.727929</td>\n",
       "      <td>-79.262029</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>M1L</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Clairlea, Golden Mile, Oakridge</td>\n",
       "      <td>43.711112</td>\n",
       "      <td>-79.284577</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>M1M</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Cliffcrest, Cliffside, Scarborough Village West</td>\n",
       "      <td>43.716316</td>\n",
       "      <td>-79.239476</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>M1N</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Birch Cliff, Cliffside West</td>\n",
       "      <td>43.692657</td>\n",
       "      <td>-79.264848</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>10</th>\n",
       "      <td>M1P</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Dorset Park, Scarborough Town Centre, Wexford ...</td>\n",
       "      <td>43.757410</td>\n",
       "      <td>-79.273304</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>11</th>\n",
       "      <td>M1R</td>\n",
       "      <td>Scarborough</td>\n",
       "      <td>Maryvale, Wexford</td>\n",
       "      <td>43.750072</td>\n",
       "      <td>-79.295849</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   PostalCode      Borough                                       Neighborhood  \\\n",
       "0         M1B  Scarborough                                     Rouge, Malvern   \n",
       "1         M1C  Scarborough             Highland Creek, Rouge Hill, Port Union   \n",
       "2         M1E  Scarborough                  Guildwood, Morningside, West Hill   \n",
       "3         M1G  Scarborough                                             Woburn   \n",
       "4         M1H  Scarborough                                          Cedarbrae   \n",
       "5         M1J  Scarborough                                Scarborough Village   \n",
       "6         M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park   \n",
       "7         M1L  Scarborough                    Clairlea, Golden Mile, Oakridge   \n",
       "8         M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West   \n",
       "9         M1N  Scarborough                        Birch Cliff, Cliffside West   \n",
       "10        M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...   \n",
       "11        M1R  Scarborough                                  Maryvale, Wexford   \n",
       "\n",
       "     Latitude  Longitude  \n",
       "0   43.806686 -79.194353  \n",
       "1   43.784535 -79.160497  \n",
       "2   43.763573 -79.188711  \n",
       "3   43.770992 -79.216917  \n",
       "4   43.773136 -79.239476  \n",
       "5   43.744734 -79.239476  \n",
       "6   43.727929 -79.262029  \n",
       "7   43.711112 -79.284577  \n",
       "8   43.716316 -79.239476  \n",
       "9   43.692657 -79.264848  \n",
       "10  43.757410 -79.273304  \n",
       "11  43.750072 -79.295849  "
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df.rename(columns={'index': 'PostalCode'}, inplace=True)\n",
    "df.head(12)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Dataframe shape:  (103, 5)\n"
     ]
    }
   ],
   "source": [
    "print(\"Dataframe shape: \", df.shape)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}